In [198]:
'''
Created on 2016. 12. 14

@author: dato
@desc: cnlp2 hw9 (HMM)
'''
#-*- coding: utf-8 -*-

import io
import numpy as np
import scipy as sp
from scipy import stats
import math

# Excercise 1 Sample Generation

In [12]:
def sample_generation(N, pi, A, B):
    
    z = []   # state
    y = []   # symbol
    ll = []   # log likelihood
        
    state = pi
    symbol = 1
    
    state_index = []
    symbol_index = []
    
    for i in range(A.shape[1]):
        state_index.append(i)
    
    for i in range(B.shape[1]):
        symbol_index.append(i)
        
    for i in range (N):
        
        # calculate state prob.
        state = state.dot(A)
        # select state according to the prob.
        state_select = stats.rv_discrete(name='custm', values=(state_index, state)).rvs(size=1)
        
        symbol = B[state_select][0]
        # select emission according to the prob.
        symbol_select = stats.rv_discrete(name='custm', values=(symbol_index, symbol)).rvs(size=1)
        
        z.append( np.asscalar( state_select ) )
        y.append( np.asscalar( symbol_select ) )         
        ll.append( np.log(  np.asscalar(state[state_select] * symbol[symbol_select])  ) )
        
    return z, y, ll

In [13]:
# initial state
pi = np.array([0.5,0.5])

#  state transition matrix
A = np.array([[0.7, 0.3],[0.9, 0.1]])

# emission prob matrix
B = np.array([[0.5,0.5],[0.2,0.8]])

In [16]:
z, y, ll = sample_generation(5, pi, A, B)
print 'sequence of state: ' + str(z)
print 'sequence of symbols: ' + str(y)
print 'log likelihood: ' + str(ll)

sequence of state: [1, 1, 0, 0, 0]
sequence of symbols: [0, 1, 1, 1, 1]
log likelihood: [-3.2188758248682006, -1.5702171992808189, -0.97816613559224252, -0.98136272861786988, -0.98072259203354395]


# Excercise 2 Forward and Backward

In [190]:
def forward(y, A, B, pi):    
    N = len(y)

    a = np.ndarray(shape=(N,len(pi)), dtype=float)
    log_a = np.ndarray(shape=(N,len(pi)), dtype=float)
    
    # initial value
    for k in range(len(pi)):
        a[0][k] = pi[k]*B[k][y[0]]
        log_a[0][k] = np.log( pi[k] ) + np.log( B[k][y[0]] )

    # loop
    # n : step, k : state (0, 1, 2, ...)
    for n in range(1, N):

        for k in range(len(pi)):

            #inner loop
            for j in range(len(pi)):
                a[n][k] += ( a[n-1][j] * A[j][k] )
                log_a[n][k] += ( a[n-1][j] * A[j][k] )
    
            a[n][k] *= B[k][y[n]]        
            log_a[n][k] = np.log( log_a[n][k] )
            log_a[n][k] += np.log(B[k][y[n]])
            
    return a, log_a

In [191]:
def backward(y, A, B, pi):   

    N = len(y) -1
    eval = 0

    b = np.ndarray(shape=(N+1,len(pi)), dtype=float)
    log_b = np.ndarray(shape=(N+1,len(pi)), dtype=float)

    # initial value
    for k in range(len(pi)):
        b[N][k] = 1
        log_b[N][k] = np.log(1)

    # # loop
    # # n : step, k : state (0, 1, 2, ...)
    for n in reversed(range(N)):

        for k in range(len(pi)):

            #inner loop
            for j in range(len(pi)):
                b[n][k] += A[k][j] * B[j][y[n+1]] * b[n+1][j]
            
            log_b[n][k] = np.log( b[n][k] )

    return b, log_b

In [192]:
# log sum_i (a_i)
# data_i = log(a_i)

def log_addition( data ) :
    result = 0
    
    for i in range (len (data)) :
        result += np.exp( data[i] -  max(data) )
    
    result = np.log( result ) 
    result += max (data)
    
    return result

In [193]:
def forward_backward(y, log_a, log_b, pi):
    
    log_a_b = []
    
    for i in range (len(pi)) :
        log_a_b.append( log_a[len(y)-1][i] + log_b[ len(y) -1][i] )
    
    return log_addition(log_a_b)

In [194]:
# initial state
pi = np.array([0.5,0.5])

#  state transition matrix
A = np.array([[0.7, 0.3],[0.9, 0.1]])

# emission prob matrix
B = np.array([[0.5,0.5],[0.2,0.8]])

# symbol
y = [0, 0, 1, 0, 0, 1, 0, 0, 0, 1]

In [195]:
a, log_a = forward(y, A, B, pi)
print '\n' + str(log_a)


[[-1.38629436 -2.30258509]
 [-2.02117263 -4.07454193]
 [-2.91830838 -3.40641095]
 [-3.38638513 -5.54557432]
 [-4.29784156 -6.16204459]
 [-5.16591642 -5.67457547]
 [-5.64300813 -7.79664416]
 [-6.55374525 -8.41846131]
 [-7.42190535 -9.31679872]
 [-8.29501576 -8.8001254 ]]


In [196]:
b, log_b = backward(y, A, B, pi)
print '\n' + str(log_b)


[[-6.81830361 -6.66811541]
 [-5.90756475 -6.04631115]
 [-5.43049475 -5.31049533]
 [-4.56234919 -4.41207475]
 [-3.65148941 -3.79116742]
 [-3.17592961 -3.05247005]
 [-2.30287514 -2.18252022]
 [-1.43422489 -1.28699216]
 [-0.52763274 -0.63487827]
 [ 0.          0.        ]]


In [197]:
ll = forward_backward(y, log_a, log_b, pi)
print ll

-7.82286480538


# Excercise 3 Gamma

In [201]:
def decoding(y, log_a, log_b):

    N = len(y) -1
    eval = 0

    r = np.ndarray(shape=(N,len(pi)), dtype=float)

    for n in range (len(r)):

        for k in range(len(pi)):

            r[n][k] = log_a[n][k] * log_b[n][k]
        
        r[n] = r[n] / sum(r[n])

    return r

In [202]:
r = decoding(y, log_a, log_b)

In [203]:
print 'sequence probability (r_nk) \n' + str(r)

sequence probability (r_nk) 
[[ 0.38104272  0.61895728]
 [ 0.32644786  0.67355214]
 [ 0.46697068  0.53302932]
 [ 0.38704642  0.61295358]
 [ 0.40183272  0.59816728]
 [ 0.48643733  0.51356267]
 [ 0.43300579  0.56699421]
 [ 0.46454121  0.53545879]
 [ 0.39833293  0.60166707]]
